In [1]:
import tensorflow as tf
# tf.enable_eager_execution()
import os

In [2]:
label_decoder = {
    "en": 0,
    "es": 1,
    "de": 2
}

In [3]:
image_directory = os.listdir('./dataset/train_spectrogram_images')
image_labels = image_directory
image_directory = ['dataset/train_spectrogram_images/' + i for i in image_directory]
images = []
labels = []
for i in image_directory:
    flag_images_dir = [(i+'/'+j) for j in os.listdir(i)]
    flag_labels_dir = [(label_decoder[i[-2:]]) for j in os.listdir(i)]
    labels = labels + flag_labels_dir
    images = images + flag_images_dir

In [4]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [432, 432])
    image /= 255.0  # normalize to [0,1] range
    return image

In [5]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [6]:
BATCH_SIZE = 16
AUTOTUNE = 1
image_count = len(images)

In [7]:
ds = tf.data.Dataset.from_tensor_slices((images, labels))

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

image_label_ds = ds.map(load_and_preprocess_from_path_label)
ds = image_label_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
ds

<DatasetV1Adapter shapes: ((?, 432, 432, 3), (?,)), types: (tf.float32, tf.int32)>

In [8]:
def change_range(image,label):
    return image, label
keras_ds = ds.map(change_range)

In [ ]:
image_batch, label_batch = next(iter(ds))

In [30]:
n_classes = 10
batch_size = 128

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float', 10)

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
               'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)
    fc = tf.reshape(conv2,[-1, 7*7*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)
    output = tf.matmul(fc, weights['out'])+biases['out']
    return output

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = prediction) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(hm_epochs):
            epoch_loss = 0
#             for _ in range(int(mnist.train.num_examples/batch_size)):
#                 epoch_x, epoch_y = mnist.train.next_batch(batch_size)
#                 _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
#                 epoch_loss += c
#             print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
#         correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
#         accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
#         print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

train_neural_network(x)